In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline

# Module 11 Lab - Model Evaluation

## Directions


The due dates for each are indicated in the Syllabus and the course calendar. If anything is unclear, please email EN685.648@gmail.com the official email for the course or ask questions in the Lab discussion area on Blackboard.

The Labs also present technical material that augments the lectures and "book".  You should read through the entire lab at the start of each module.

<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
Please follow the directions and make sure you provide the requested output. Failure to do so may result in a lower grade even if the code is correct or even 0 points.
</div>

1. Show all work/steps/calculations using Code and Markdown cells.
2. Submit your notebook (.ipynb).
3. You may use any core Python libraries or Numpy/Scipy. **Additionally, code from the Module notebooks and lectures is fair to use and modify.** You may also consult Stackoverflow (SO). If you use something from SO, please place a comment with the URL to document the code.

In [8]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import random
import patsy
import sklearn.linear_model as linear
import models

sns.set(style="whitegrid")
# load whatever other libraries you need including models.py

## Model Evaluation and Improvement

As we saw in both the Linear Regression and Logistic Regression modules, there is a Statistician's view of Model Evaluation (and perhaps, Improvement) and a Machine Learning view of Model Evaluation and Improvement.

We'll be working with the **insurance data**.

**1. Load the data, perform your transformations, and using the Bootstrap version of the Linear Regression function, estimate your final model from Lab 10 and show the Bootstrap results** (You can also use the final model from the Solution if you like).

Here we load in the data and make sure it loads in correctly.

In [14]:
insurance = pd.read_csv("https://raw.githubusercontent.com/fundamentals-of-data-science/datasets/master/insurance.csv", header=0)

In [15]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [7]:
insurance.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


It looks good to me. I'll be using the final model from the Lab 10 Solution. We had a model where `charges` was the target variable and our features included `age_sq`, `male`, `bmi`, `smoke_yes`, `smoke_yes:bmi`, `smoke_yes:bmi_above_30`, and `children`.

Let's make sure we properly transform our variables correctly so we can use them in our model. Age is easy to square so we do that first, and can create the one hot encodings for our categorical variables sex -> male and smoker -> smokes_yes. Then we just apply a lambda function to get an encoding for bmi over 30 and we should be set.

In [16]:
insurance['age_sq'] = insurance['age'] ** 2

insurance = pd.concat([insurance, pd.get_dummies(insurance["sex"])], axis=1)
insurance = pd.concat([insurance, pd.get_dummies(insurance["smoker"], prefix="smoke")], axis=1)

insurance["bmi_above_30"] = insurance.bmi.apply(lambda bmi: 1 if bmi > 30.0 else 0)

Let's check our variables are in the dataframe

In [17]:
insurance.head()

,age,sex,bmi,children,smoker,region,charges,age_sq,female,male,smoke_no,smoke_yes,bmi_above_30
0,19,female,27.900,0,yes,southwest,16884.92400,361,True,False,False,True,0
1,18,male,33.770,1,no,southeast,1725.55230,324,False,True,True,False,1
2,28,male,33.000,3,no,southeast,4449.46200,784,False,True,True,False,1
3,33,male,22.705,0,no,northwest,21984.47061,1089,False,True,True,False,0
4,32,male,28.880,0,no,northwest,3866.85520,1024,False,True,True,False,0


Looks good to me. Now we can recreate this model and use bootstrap for linear regression from our models.py file.

In [32]:
model = "charges ~ age_sq + male + bmi + smoke_yes + smoke_yes:bmi + bmi_above_30 + smoke_yes:bmi_above_30 + children"
final = models.bootstrap_linear_regression(model, data=insurance)
models.describe_bootstrap_lr(final)

,,,95% BCI</strong,
Coefficients</strong,,Mean</strong,Lo</strong,Hi</strong
,$\beta_{0}$,2169.21,443.46,3814.12
age_sq,$\beta_{1}$,-517.49,-1004.94,-25.80
male,$\beta_{2}$,1479.18,-1798.49,5397.40
bmi,$\beta_{3}$,3.34,3.14,3.57
smoke_yes,$\beta_{4}$,-3.22,-71.88,56.29
smoke_yes:bmi,$\beta_{5}$,471.11,329.07,607.69
bmi_above_30,$\beta_{6}$,102.95,-768.90,1102.26
smoke_yes:bmi_above_30,$\beta_{7}$,15122.63,13151.61,17233.83
children,$\beta_{8}$,651.45,482.53,792.05


Note I include bmi_above_30 as a separate variable here because the `bootstrap_linear_regression` function was not working without including it in the model before the interaction term, though it's not necessary as we see the 95% BCI contains 0 and it may have an unexpected sign.

Here we have the estimates of the coefficients in the model for the bootstrap. We see an $R^2$ of 87% and a $\sigma$ of 4380.25. Which is about a third of the original as we can see below

In [42]:
print('Original Std: ', '%.2f' % np.std(insurance['charges']))
print('Model Sigma: ', '%.2f' % final['sigma'])

Original Std:  12105.48
Model Sigma:  4380.25


**2. Perform three rounds of 10-fold cross validation, estimating $R^2$ and $\sigma$ each round. Using the results for the test data, calculate 95% Bootstrap estimates of the credible intervals for each.** Comment on these intervals and the intervals from above. Are the average values different? Are the intervals different?

**3. Using Learning Curves and $\sigma$ as your evaluation metric determine if more data will improve the estimation of the model.**

**4. It was shown that `age_sq` improved the performance of the model. Perhaps a different polynomial would have been better. Generate Validation Curves for $age_n$ where `n` = [1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5] and select the best transformation.**

**5. Using Ridge Regression to estimate a model for the insurance data. Compare it with your final Linear Regression model.** (If you get far ahead, you may need to write your own function. Here are the sklearn docs: http://scikit-learn.org/stable/modules/linear_model.html)